### Goals

0. We need to map velocities to the atmosphere somehow. what parameterizations are "simple"?

  * fixed/solid body rotation
  * differential rotation

1. $K_p-v_{\rm sys}$ diagram for a planet (for now we'll use a stellar spectrum) that is orbiting a star, with a velocity field in its atmosphere

2. estimate the effect of a velocity field throughout the terminator on the transmission spectrum

3. compare GCM velocity fields with observations (in transit transmission and in full-orbit CCFs)

In [ ]:
%matplotlib inline

import astropy.units as u
import matplotlib.pyplot as plt
from astropy.modeling.models import BlackBody
from expecto import get_spectrum
import numpy as np

Set up the lon ($\phi$) and lat ($\theta$) grids:

In [ ]:
n_lats = 7
n_lons = 25
phi = np.linspace(0, 2*np.pi, n_lons)

lat_frac = 0.9
theta = np.arcsin(np.linspace(-lat_frac, lat_frac, n_lats))

Rp = 1 * u.R_jup

phi2d, theta2d = np.meshgrid(
    phi, theta, 
)

$$ v(\phi, \theta) = \sin(\theta) $$

In [ ]:
colat_to_lat = 0 #- np.pi / 2
delta_phi = 0 
x = Rp * np.cos(theta2d) * np.sin(phi2d)
y = Rp * np.sin(theta2d)
z = Rp * np.sin(theta2d) * np.sin(phi2d) #* np.sign(theta2d + colat_to_lat)

dx = - np.sin(phi2d - delta_phi)
dy = np.cos(phi2d)
dz = np.zeros_like(phi2d)

plt.scatter(x.value.ravel(), y.value.ravel(), c=-dx.ravel())
plt.colorbar()
plt.gca().set(
    xlabel='x', 
    ylabel='y'
)
plt.gca().set_aspect('equal')

In [ ]:
from specutils import SpectralRegion, Spectrum1D
from specutils.manipulation import extract_region
spec = get_spectrum(2300, 5.0, cache=True)

In [ ]:
minimum, maximum = 0.4*u.um, 0.402*u.um

mask = (
    (spec.wavelength > minimum) & 
    (spec.wavelength < maximum)
)

wavelength = spec.wavelength[mask]
flux_cube = np.squeeze(spec.flux[mask][:, None, None] * np.ones_like(x)[None, None, :])

print(flux_cube.shape)

The shape of the resulting `flux_cube` is (wavelength, lat, lon): $N_\lambda, N_\theta, N_\phi$

Now let's start with a fixed velocity map:

In [ ]:
eq_velocity = 100 * u.km/u.s
pole_velocity = 5 * u.km/u.s
alpha = float((eq_velocity - pole_velocity) / pole_velocity)

# this is the variation of wind velocity with latitude, written in
# the form often used to describe stellar differential rotation:
velocity_map = pole_velocity * (1 + alpha * np.cos(theta2d))
radial_velocity_map = velocity_map * -dx

# here's the velocity map with latitude:
plt.plot(np.degrees(theta), velocity_map[:, 0]);
plt.gca().set(
    xlabel='lat',
    ylabel='local velocity [km/s]'
);

Compute the "visibility" of each spatial pixel on the sphere. `1` is a pixel viewed face-on, `0` is a pixel that is not visible.

In [ ]:
visibility = np.cos(phi2d - np.pi) * np.sin(theta2d + np.pi/2)**2
visibility[visibility < 0] = 0

Plot them all:

In [ ]:
fig = plt.figure(figsize=(20, 4))
ax0 = fig.add_subplot(131, projection='mollweide')
ax1 = fig.add_subplot(132, projection='mollweide')
ax2 = fig.add_subplot(133, projection='mollweide')
cax = ax0.pcolormesh(phi - np.pi, theta, velocity_map)
plt.colorbar(cax, ax=ax0, label='local velocity [km/s]')
cax = ax1.pcolormesh(phi - np.pi, theta, visibility)
plt.colorbar(cax, ax=ax1, label='fractional visibility')
cax = ax2.pcolormesh(phi - np.pi, theta, radial_velocity_map)
plt.colorbar(cax, ax=ax2, label='radial velocity [km/s]')

cax = ax2.contour(phi - np.pi, theta, visibility, levels=[0.1, 0.3, 0.8], cmap=plt.cm.Greys_r)

ax0.set_title('local velocity')
ax1.set_title('visibility')
ax2.set_title('map=radial velocity;\ncontour=visibility')
plt.show()

Now for each spatial pixel on the sphere in $\theta, \phi$, compute the redshifts/blueshifts for each wavelength at each pixel:

In [ ]:
wavelength_cube = radial_velocity_map[None, ...].to(u.nm, u.doppler_optical(wavelength[:, None, None]))

Take the flux cube (wave, lat, lon) and scale it by the visibility of each spatial pixel on the sphere:

In [ ]:
weighted_flux_cube = flux_cube * visibility[None, ...] # weighting the spectrum by the visibility

The weighted flux cube has doppler shifted wavelengths, but the observer only sees a single set of wavelengths (named here `wavelength`). Let's interpolate the doppler shifted spectra onto each observer-frame wavelength, at each spatial pixel:

In [ ]:
# We're going to take the 3D cube, and make it 2D temporarily so we can interpolate efficiently:
new_shape = (weighted_flux_cube.shape[0], np.prod(weighted_flux_cube.shape[1:]))

# loop over the (now single) spatial dimension and interpolate in the wavelength dimension
shifted_weighted_flux_cube = np.empty(new_shape)
for i in range(shifted_weighted_flux_cube.shape[1]):
    shifted_weighted_flux_cube[:, i] = np.interp(
        wavelength.to(u.um).value, 
        wavelength_cube.reshape(new_shape)[:, i].to(u.um).value, 
        weighted_flux_cube.reshape(new_shape)[:, i].value
    )

# now put the result back into the 3D shape (wave, lat, lon):
shifted_weighted_flux_cube = shifted_weighted_flux_cube.reshape(weighted_flux_cube.shape)

In [ ]:
# the observed spectrum of the visible hemisphere is: 
observed_spectrum = shifted_weighted_flux_cube.sum(axis=(1, 2))

In [ ]:
# this normalization factor shifts a single spatial element's spectrum up to the same scale
# as the observed spectrum of a full hemisphere:
norm = spec.flux[mask].max() / observed_spectrum.max()
plt.plot(wavelength, spec.flux[mask] / norm, label='intrinsic local spectrum')
plt.plot(wavelength, observed_spectrum, label='observed hemispheric spectrum')
plt.legend()